# Craigslist Scraping Operationalization

## Raw Data Examples

SQL Query Results by Foreign Key 'Zip Code' for: 

1. Count of Businesses in Same Zip Code
2. Count of Eviction Notices in same Zip Code ignore
3. Count of Different Types of Schools in same Zip Code
4. Avg Cost of Home Prices


In [1]:
static_evaluations_by_zip = {'businesses': {'94102': 1, '11111': 5, '94104': 10, '94105': 10 }, 
             'evictions': {'94102': 7, '11111': 4, '94104': 10, '94105': 9 },
             'schools_k9': {'94102': 10, '11111': 5, '94104': 2, '94105': 8 },
             'schools_hs': {'94102': 9, '11111': 6, '94104': 6, '94105': 10 },
             'home_prices': {'94102': 8, '11111': 4, '94104': 1, '94105': 4 }}

Raw Hive Table for each of: 

1. Bicycle Parking Locations
2. Bicycle Share Locations
3. Parking Lots / Spaces
4. SFPD Incidents 2016
5. Trees

with GeoTag Foreign Key. 

In [2]:
bike_parking = {'location 1': (37.7606289177, -122.410647009), 
                'location 2': (37.7855355791102, -122.411302813025),
                'location 3': (37.7759676911831, -122.441396661871),
                'location 4': (37.7518243814, -122.426627114),
                'location 5': (37.75182438, -122.4266271)}

bike_share =   {'location 1': (37.7606289177, -122.410647009), 
                'location 2': (37.7855355791102, -122.411302813025),
                'location 3': (37.7759676911831, -122.441396661871),
                'location 4': (37.7518243814, -122.426627114),
                'location 5': (37.75182438, -122.4266271)}

off_street_parking =      {'location 1': ["Private", 13, (37.7606289177, -122.410647009)], 
                'location 2': ["Private", 15, (37.7855355791102, -122.411302813025)],
                'location 3': ["Public", 18, (37.7759676911831, -122.441396661871)],
                'location 4': ["Private", 27, (37.7518243814, -122.426627114)],
                'location 5': ["Private", 60, (37.75182438, -122.4266271)]}

SFPD =      {'location 1': (37.7606289177, -122.410647009), 
                'location 2': (37.7855355791102, -122.411302813025),
                'location 3': (37.7759676911831, -122.441396661871),
                'location 4': (37.7518243814, -122.426627114),
                'location 5': (37.75182438, -122.4266271)}

trees =      {'location 1': (37.7606289177, -122.410647009), 
                'location 2': (37.7855355791102, -122.411302813025),
                'location 3': (37.7759676911831, -122.441396661871),
                'location 4': (37.7518243814, -122.426627114),
                'location 5': (37.75182438, -122.4266271)}

List of Zip Codes (delete?)

In [3]:
zipcodes = [94102,94103,94104,94105,94107,94108,94109,94110,94111,94112,94114,94115,94116,94117,94118,94119,94120,
            94121,94122,94123,94124,94125,94126,94127,94128,94129,94130,94131,94132,94133,94134,94137,94139,94140,
            94141,94142,94143,94144,94145,94146,94147,94151,94158,94159,94160,94161,94163,94164,94172,94177,94188]

### Sample Inputs from User

In [4]:
max_rent = 4000
min_rent = 1000
category = 'apa'
min_rank_businesses = None
min_rank_evictions = 1
min_rank_schools_k9 = 8
min_rank_schools_hs = None
min_rank_home_prices = 5
close_to_bike_parking = "Yes"       # Care about close bike parking, Yes or No
distance_to_bike_share = "Short"        # Short, Medium, Long
density_of_offstreet_parking = "Medium"   # Low, Medium, High Density within Xkm (Select Low if you don't care)
density_of_SFPD_Incidents = "Low"       # Low, Medium, High Density in 2016
density_of_trees_100m = "High"          # Low, Medium, High Density within 500m

### Sample Raw Outputs from Craigslist Scraper

In [5]:
output = {'id': '6060895324', 'has_map': True, 'price': '$1600', 'url': 'http://sfbay.craigslist.org/sfc/apa/6060895324.html',
          'name': 'Furnished Room', 'has_image': True, 'datetime': '2017-03-26 09:33', 'where': 'nob hill', 'geotag': (37.790788, -122.419036)}


# Scraper Class w/ Filters

In [6]:
def scrape_craigslist(max_rent= None, min_rent = None, cat = category):

    from craigslist import CraigslistHousing
    import filtering_functions
    import zip_lookup
    import neighbourhood_lookup
    import math
    from IPython.display import Markdown, display
    def printmd(string):
        display(Markdown(string))
    

    cl = CraigslistHousing(site='sfbay', area='sfc', category= cat,
                             filters={'max_price': max_rent, 'min_price': min_rent})

    results = cl.get_results(sort_by='newest', geotagged=True, limit=100) #do we need to set a reasonable limit?
    tentative_rental = []
    valid_rentals = []
    for result in results:
        print("Evaluating: ", result['name'], result['url'])
        ################################################################################################
        ### INITIALIZE RESULT
        
        # assign geotag if it is provided in the rental ad
        if result['geotag'] is not None:
            geotag = result['geotag']
        else:
            continue #skip for now because there is no geotag    
        
        zipcode = zip_lookup.zip_lookup_by_geotag(geotag)
        tentative_rental.append(result)
        
        ### Get Approximate Neighbourhood by Geotag
        if result["where"] != None:
            result["area"] = result["where"]
        else:
            result["area"] = neighbourhood_lookup.neighbourhood_lookup(geotag)
        
        
        ################################################################################################
        ### APPLY FILTERS
        
        
        #-----------------------------------------------------------------------------------------------
        ## ZIPCODE BASED FILTERS
        #-----------------------------------------------------------------------------------------------
        
        # # OF BUSINESSES
        if min_rank_businesses == None:
            pass
        else:
            if zipcode not in filtering_functions.check_businesses(min_rank_businesses, static_evaluations_by_zip):
                continue #doesn't meet user criteria, go on to next result
        
        # NEARBY SCHOOLS
        if min_rank_evictions == None:
            pass
        else:
            if zipcode not in filtering_functions.check_evictions(min_rank_evictions, static_evaluations_by_zip):
                continue #doesn't meet user criteria, go on to next result
        
        #-----------------------------------------------------------------------------------------------
        ## DISTANCE BASED FILTERS
        #-----------------------------------------------------------------------------------------------
        
        #----------------------------------
        ## DISTANCE TO X FILTERS
        #----------------------------------
        
        # BIKE PARKING (is bike parking within 20km of the rental? (huge window until we get actual locations loaded))
        if close_to_bike_parking == "No":
            pass #skip this filter, user doesn't care
        else:
            BP_close, BP_location, BP_distance = filtering_functions.close_to_bike_parking(geotag, bike_parking)
            if BP_close == False:
                continue #doesn't meet user criteria, go on to next result
            else:
                result["BP_close"] = BP_close
                result["BP_location"] = BP_location
                result["BP_distance"] = BP_distance
        
        
        #----------------------------------
        ## DENSITY STYLE FILTERS
        #----------------------------------
        
        # VEHICLE PARKING DENSITY (Initial Parking window set @ 20km, huge window until we get actual locations loaded)
        if density_of_offstreet_parking == "Low":
            pass #skip this filter, user doesn't care
        else:
            Public_Parking_Density, Public_Parking_Spots, Private_Parking_Density, \
                Private_Parking_Spots = filtering_functions.parking_density(geotag,off_street_parking)
            if (Public_Parking_Density == "High Public Parking Density" or Private_Parking_Density == "High Private Parking Density") and density_of_offstreet_parking == "High":
                result["Public_Parking_Density"], result["Public_Parking_Spots"], result["Private_Parking_Density"], \
                result["Private_Parking_Spots"] = filtering_functions.parking_density(geotag,off_street_parking)
            elif (Public_Parking_Density in ("High Public Parking Density","Medium Public Parking Density") \
                or Private_Parking_Density in ("High Private Parking Density","Medium Private Parking Density")) \
                and density_of_offstreet_parking == "Medium":
                result["Public_Parking_Density"], result["Public_Parking_Spots"], result["Private_Parking_Density"], \
                result["Private_Parking_Spots"] = filtering_functions.parking_density(geotag,off_street_parking)
            else:
                continue #doesn't meet user criteria, go on to next result
        
        
        

        ################################################################################################
        # Made it to the end of the filters intact? Rental is Valid for this User's Query!
        valid_rentals.append(result)
        tentative_rental = [] # reset the tentative rental, continue loop
        
        ################################################################################################
        ### DISPLAY VALID RESULTS
        printmd('**-------------------VALID RESULT------------------**')
        print("Area: ", result["area"], "\n",
              "Price: ", result["price"], "\n",
              "Listing Name: ", result["name"], "\n",
              "URL: ", result["url"], "\n",
              "Bike Parking (Close?, Location Name, Distance (km)): ",result["BP_close"], result["BP_location"], result["BP_distance"], "\n",
              "Local Public Off-Street Parking Density, # of Spots: ", result["Public_Parking_Density"], result["Public_Parking_Spots"], "\n",
              "Local Private Off-Street Parking Density, # of Spots: ",result["Private_Parking_Density"], result["Private_Parking_Spots"])
        printmd('**-------------------------------------------------**')
        
        
    return valid_rentals


In [7]:
scrape_craigslist(max_rent = max_rent, min_rent = min_rent)

Evaluating:  Leasing For Spring! 3 Weeks Free Rent, 6 Months Free Parking! http://sfbay.craigslist.org/sfc/apa/6080088197.html


**-------------------VALID RESULT------------------**

Area:  SOMA / south beach 
 Price:  $3420 
 Listing Name:  Leasing For Spring! 3 Weeks Free Rent, 6 Months Free Parking! 
 URL:  http://sfbay.craigslist.org/sfc/apa/6080088197.html 
 Bike Parking (Close?, Location Name, Distance (km)):  True location 2 1.3258665132557403 
 Local Public Off-Street Parking Density, # of Spots:  Low Public Parking Density 18 
 Local Private Off-Street Parking Density, # of Spots:  High Private Parking Density 115


**-------------------------------------------------**

Evaluating:  ELLIS STREET ONE BEDROOM http://sfbay.craigslist.org/sfc/apa/6079984552.html
Evaluating:  One of a Kind Studio Loft in Showplace Square! One Month FREE! http://sfbay.craigslist.org/sfc/apa/6080083074.html
Evaluating:  Historic and Handsome Home with Exposed Brick - Lower Nob Hill http://sfbay.craigslist.org/sfc/apa/6080071194.html


...

Evaluating:  Corner Apt. Custom Finishes with Modern Design! Free Wifi/Cable http://sfbay.craigslist.org/sfc/apa/6079869003.html
Evaluating:  Luxury Studios Surrounded By A Beautiful New Park at 33 8th Street! http://sfbay.craigslist.org/sfc/apa/6079854949.html


**-------------------VALID RESULT------------------**

Area:  SOMA / south beach 
 Price:  $2550 
 Listing Name:  Luxury Studios Surrounded By A Beautiful New Park at 33 8th Street! 
 URL:  http://sfbay.craigslist.org/sfc/apa/6079854949.html 
 Bike Parking (Close?, Location Name, Distance (km)):  True location 2 0.8371889954220105 
 Local Public Off-Street Parking Density, # of Spots:  Low Public Parking Density 18 
 Local Private Off-Street Parking Density, # of Spots:  High Private Parking Density 115


**-------------------------------------------------**

[{'BP_close': True,
  'BP_distance': 1.3258665132557403,
  'BP_location': 'location 2',
  'Private_Parking_Density': 'High Private Parking Density',
  'Private_Parking_Spots': 115,
  'Public_Parking_Density': 'Low Public Parking Density',
  'Public_Parking_Spots': 18,
  'area': 'SOMA / south beach',
  'datetime': '2017-04-08 13:59',
  'geotag': (37.773717, -122.409303),
  'has_image': True,
  'has_map': True,
  'id': '6080088197',
  'name': 'Leasing For Spring! 3 Weeks Free Rent, 6 Months Free Parking!',
  'price': '$3420',
  'url': 'http://sfbay.craigslist.org/sfc/apa/6080088197.html',
  'where': 'SOMA / south beach'},
 {'BP_close': True,
  'BP_distance': 0.6729616185043035,
  'BP_location': 'location 2',
  'Private_Parking_Density': 'High Private Parking Density',
  'Private_Parking_Spots': 115,
  'Public_Parking_Density': 'Low Public Parking Density',
  'Public_Parking_Spots': 18,
  'area': 'lower nob hill',
  'datetime': '2017-04-08 13:54',
  'geotag': (37.789046, -122.417541),
  '